In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess

In [4]:
train_results_dir = os.getcwd() + "/../002-Training-models/train_results/"
!ls {train_results_dir}

model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_weapons
model_yolov5_tf_yolov5s_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_weapons
model_yolov7_tf_yolov7training_imgsz_800_epochs_100_batch_8_dataset_v1_weapons
model_yolov8_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8l_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8m_imgsz_640_epochs_73_batch_16_dataset_v2
model_yolov8_tf_yolov8s_imgsz_640_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v1_weapons
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v3_randomizedclips
model_yolov8_tf_yolov8s_imgsz_800_epochs_78_batch_16_dataset_v2


In [5]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"
output_file = "eval_vs_original_dataset.txt"

In [6]:
dataset_dir

'/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/../datasets/'

In [8]:
%cd {eval_results_dir}

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models


In [9]:
# Download YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

%cd {eval_results_dir}
!mkdir yolov8

from ultralytics import YOLO
print('YOLOv8 setup complete.')

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models
mkdir: yolov8: File exists
YOLOv8 setup complete.


In [10]:
# Download YOLOv7
%cd {eval_results_dir}
# !git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

from IPython import display
display.clear_output()
print('YOLOv7 setup complete.')

YOLOv7 setup complete.


In [11]:
# Download YOLOv5
%cd {eval_results_dir}
# !git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython import display
display.clear_output()
print('YOLOv5 setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

YOLOv5 setup complete. Using torch 2.0.0 CPU


In [13]:
%cd {eval_results_dir}

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models


# Evaluate vs original datasets

In [14]:
output_file

'eval_vs_original_dataset.txt'

In [15]:
for folder_name in sorted(os.listdir(train_results_dir)):
    if not folder_name.startswith("model"):
        continue

    folder_path = os.path.join(train_results_dir, folder_name, "weights")

    # Split the folder name into variables
    variables = folder_name.split('_')

    # Extract the required variables
    model = variables[1]
    tf = variables[3]
    imgsz = variables[5]
    epochs = variables[7]
    batch = variables[9]
    dataset = variables[11]
    
    print(folder_name)
    print("Model:", model)
    print("TF:", tf)
    print("Imgsz:", imgsz)
    print("Epochs:", epochs)
    print("Batch:", batch)
    print("Dataset:", dataset)
    print("Executing command...")
    print()
    
    if dataset == "v1":
        dataset_location = dataset_dir + model + "_original"
    elif dataset == "v2":
        dataset_location = dataset_dir + model + "_original_v2"
        
        
    command = None
    
    if model == "yolov5":
        %cd yolov5
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"python3 val.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task val --name {folder_name}"
        print(command)

    elif model == "yolov7":
        %cd yolov7
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"python3 test.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task val --name {folder_name}"
        print(command)

    elif model == "yolov8" and dataset != "v3":
        %cd yolov8
        best_pt_path = os.path.join(folder_path, "best.pt")
        command = f"yolo val model={best_pt_path} data={dataset_location}/data.yaml split=test name={folder_name}"
        print(command)

    if command:
        # Run the command using subprocess.Popen and capture the output
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
            # Open the output file for writing
            with open(f"{eval_results_dir}/{output_file}", "a") as file:
                !ls {eval_results_dir}/{output_file}
                file.write(folder_name)
                file.write(f"\nModel: {model}")
                file.write(f"\nTF: {tf}")
                file.write(f"\nImgsz: {imgsz}")
                file.write(f"\nEpochs: {epochs}")
                file.write(f"\nBatch: {batch}")
                file.write(f"\nDataset: {dataset}\n")
                # Read and display each line of the output as it becomes available
                for line in process.stdout:
                    print(line, end="")
                    file.write(line)
        %cd ..

    from IPython import display
    display.clear_output()
    

    print("Executing command...", "DONE!")
    print()

Executing command... DONE!



In [16]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"

def process_file(file_path):
    results = list()
    curr_res = dict()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Model:"):
            curr_res["model"] = line.split()[1]
        elif line.startswith("TF:"):
            curr_res["tf"] = line.split()[1]
        elif line.startswith("Imgsz:"):
            curr_res["imgsz"] = line.split()[1]
        elif line.startswith("Epochs:"):
            curr_res["epochs"] = line.split()[1]
        elif line.startswith("Batch:"):
            curr_res["batch"] = line.split()[1]
        elif line.startswith("Dataset:"):
            curr_res["dataset"] = line.split()[1]
            print(line.split())
            
        if  "saved" in line or "all" in line or "knife" in line or "pistol" in line:
            if line.strip(" ").startswith("all"):
                P, R, mAP5, mAP95 = line.split()[3:]
                curr_res["all_P"] = P
                curr_res["all_R"] = R
                curr_res["all_F1"] = (float(P) * float(R)) / (float(P) + float(R))
                curr_res["all_mAP@.5"] = mAP5
                curr_res["all_mAP@.5:.95"] = mAP95

            elif line.strip(" ").startswith("knife"):
                P, R, mAP5, mAP95 = line.split()[3:]
                curr_res["knife_P"] = P
                curr_res["knife_R"] = R
                curr_res["knife_F1"] = (float(P) * float(R)) / (float(P) + float(R))
                curr_res["knife_mAP@.5"] = mAP5
                curr_res["knife_mAP@.5:.95"] = mAP95

            if line.strip(" ").startswith("all"): # or line.strip(" ").startswith("gun"):
                P, R, mAP5, mAP95 = line.split()[3:]
                curr_res["pistol_P"] = P
                curr_res["pistol_R"] = R
                curr_res["pistol_F1"] = (float(P) * float(R)) / (float(P) + float(R))
                curr_res["pistol_mAP@.5"] = mAP5
                curr_res["pistol_mAP@.5:.95"] = mAP95

            elif line.strip(" ").startswith("Results saved to"):
                model_key = line.split("/")[-1].replace('\x1b[0m', '').replace('\n', '')
                model_info = model_key.split("_")
                    
                curr_res["model_key"] = model_key

                results.append(curr_res)
                curr_res = dict()

    df = pd.DataFrame(results)
    df.set_index('model_key', inplace=True)
    return df

file_path = os.path.join(eval_results_dir, output_file)
df = process_file(file_path)
df.to_csv(f'{eval_results_dir}/eval_vs_original_dataset.csv')
df

['Dataset:', 'v1']
['Dataset:', 'v1']
['Dataset:', 'v1']
['Dataset:', 'v1']
['Dataset:', 'v1']
['Dataset:', 'v1']
['Dataset:', 'v1']
['Dataset:', 'v2']
['Dataset:', 'v1']
['Dataset:', 'v1']
['Dataset:', 'v2']


,model,tf,imgsz,epochs,batch,dataset,all_P,all_R,all_F1,all_mAP@.5,...,pistol_P,pistol_R,pistol_F1,pistol_mAP@.5,pistol_mAP@.5:.95,knife_P,knife_R,knife_F1,knife_mAP@.5,knife_mAP@.5:.95
model_key,,,,,,,,,,,,,,,,,,,,,
model_yolov5_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_weapons,yolov5,no,800,100,16,v1,0.828,0.737,0.389927,0.8,...,0.828,0.737,0.389927,0.8,0.467,0.881,0.824,0.425774,0.891,0.547
model_yolov5_tf_yolov5s_imgsz_640_epochs_100_batch_16_dataset_v1_weapons,yolov5,yolov5s,640,100,16,v1,0.844,0.822,0.416427,0.857,...,0.844,0.822,0.416427,0.857,0.522,0.903,0.875,0.444390,0.922,0.59
model_yolov5_tf_yolov5s_imgsz_800_epochs_100_batch_16_dataset_v1_weapons,yolov5,yolov5s,800,100,16,v1,0.852,0.796,0.411524,0.831,...,0.852,0.796,0.411524,0.831,0.517,0.913,0.869,0.445228,0.908,0.581
model_yolov7_tf_yolov7training_imgsz_640_epochs_100_batch_16_dataset_v1_weapons,yolov7,yolov7training,640,100,16,v1,0.886,0.901,0.446719,0.91,...,0.886,0.901,0.446719,0.91,0.649,0.938,0.942,0.469998,0.968,0.677
model_yolov7_tf_yolov7training_imgsz_800_epochs_100_batch_8_dataset_v1_weapons,yolov7,yolov7training,800,100,8,v1,0.883,0.847,0.432313,0.893,...,0.883,0.847,0.432313,0.893,0.616,0.922,0.882,0.450778,0.956,0.654
model_yolov8_tf_no_imgsz_800_epochs_100_batch_16_dataset_v1_weapons,yolov8,no,800,100,16,v1,0.871,0.776,0.410380,0.846,...,0.871,0.776,0.410380,0.846,0.599,0.92,0.861,0.444761,0.925,0.657
model_yolov8_tf_yolov8l_imgsz_800_epochs_100_batch_16_dataset_v1_weapons,yolov8,yolov8l,800,100,16,v1,0.867,0.758,0.404422,0.838,...,0.867,0.758,0.404422,0.838,0.597,0.916,0.861,0.443824,0.919,0.655
model_yolov8_tf_yolov8m_imgsz_640_epochs_73_batch_16_dataset_v2,yolov8,yolov8m,640,73,16,v2,0.993,0.962,0.488627,0.976,...,0.993,0.962,0.488627,0.976,0.941,NaN,NaN,NaN,NaN,NaN
model_yolov8_tf_yolov8s_imgsz_640_epochs_100_batch_16_dataset_v1_weapons,yolov8,yolov8s,640,100,16,v1,0.888,0.777,0.414400,0.86,...,0.888,0.777,0.414400,0.86,0.623,0.927,0.833,0.438745,0.927,0.661
